In [30]:
import pandas as pd
import numpy as np
import dedupe

In [31]:
df_kun = pd.read_excel('data/Konzerndaten.xlsx', 3, engine='openpyxl')
df_kundengruppe = pd.read_excel('data/Konzerndaten.xlsx', 4, engine='openpyxl')

In [32]:
df_kundengruppe.columns = ['KD-Grp', 'KD-Grp-Name', 'VKL']
df_kun = pd.merge(df_kun, df_kundengruppe, how='left', on='KD-Grp')
grps = df_kun['KD-Grp'].unique()

In [33]:
df_kun.Ort = df_kun.Ort.apply(str)
ks = df_kun.where(df_kun.notnull(), None).to_dict(orient='index')


In [34]:
len(df_kun)

15890

In [35]:
fields = [
    {'field': 'Name', 'type': 'String'},
    {'field': 'Str', 'type': 'String', 'has missing': True},
    {'field': 'PLZ', 'type': 'Exact', 'has missing': True},
    {'field': 'Ort', 'type': 'String', 'has missing': True},
    {'field': 'Anz-Mitarb', 'type': 'Price', 'has missing': True},
    {'field': 'Konzern', 'type': 'Exact', 'has missing': True},
    {'field': 'KD-Grp', 'type': 'Categorical', 'has missing': True
        , 'categories': grps},
]

In [12]:
deduper = dedupe.Dedupe(fields)

# first run
# deduper.prepare_training(ks)
# dedupe.console_label(deduper)

# with existing training data
with open('training.json') as f: 
    deduper.prepare_training(ks, training_file=f) 


INFO:dedupe.api:reading training from file
INFO:dedupe.canopy_index:Removing stop word  1
INFO:dedupe.canopy_index:Removing stop word be
INFO:dedupe.canopy_index:Removing stop word er
INFO:dedupe.canopy_index:Removing stop word r 
INFO:dedupe.canopy_index:Removing stop word st
INFO:dedupe.canopy_index:Removing stop word  S
INFO:dedupe.canopy_index:Removing stop word St
INFO:dedupe.canopy_index:Removing stop word ge
INFO:dedupe.canopy_index:Removing stop word de
INFO:dedupe.canopy_index:Removing stop word el
INFO:dedupe.canopy_index:Removing stop word aß
INFO:dedupe.canopy_index:Removing stop word ra
INFO:dedupe.canopy_index:Removing stop word ße
INFO:dedupe.canopy_index:Removing stop word en
INFO:dedupe.canopy_index:Removing stop word eg
INFO:dedupe.canopy_index:Removing stop word ei
INFO:dedupe.canopy_index:Removing stop word an
INFO:dedupe.canopy_index:Removing stop word he
INFO:dedupe.canopy_index:Removing stop word  2
INFO:dedupe.canopy_index:Removing stop word le
INFO:dedupe.canop

In [13]:
deduper.train() # when not yet trained

INFO:rlr.crossvalidation:using cross validation to find optimum alpha...
INFO:rlr.crossvalidation:optimum alpha: 0.100000, score 0.5727715091983886
INFO:dedupe.training:Final predicate set:
INFO:dedupe.training:(SimplePredicate: (commonSixGram, Name), SimplePredicate: (twoGramFingerprint, Str))
INFO:dedupe.training:(SimplePredicate: (wholeFieldPredicate, Konzern), TfidfNGramCanopyPredicate: (0.4, Name))
INFO:dedupe.training:(SimplePredicate: (sortedAcronym, Str), SimplePredicate: (wholeFieldPredicate, Name))
INFO:dedupe.training:(SimplePredicate: (commonThreeTokens, Name), SimplePredicate: (fingerprint, Name))


In [14]:
# write down new model
with open('training.json', 'w') as f:
    deduper.write_training(f)
with open('learned_settings', 'wb') as f:
    deduper.write_settings(f)

In [36]:
# load saved model
with open('learned_settings', 'rb') as f:
    deduper = dedupe.StaticDedupe(f)

INFO:dedupe.api:((SimplePredicate: (commonSixGram, Name), SimplePredicate: (twoGramFingerprint, Str)), (SimplePredicate: (wholeFieldPredicate, Konzern), TfidfNGramCanopyPredicate: (0.4, Name)), (SimplePredicate: (sortedAcronym, Str), SimplePredicate: (wholeFieldPredicate, Name)), (SimplePredicate: (commonThreeTokens, Name), SimplePredicate: (fingerprint, Name)))


In [37]:
clustered_dupes = deduper.partition(ks, 0.33)

INFO:dedupe.canopy_index:Removing stop word  A
INFO:dedupe.canopy_index:Removing stop word Ge
INFO:dedupe.canopy_index:Removing stop word an
INFO:dedupe.canopy_index:Removing stop word ch
INFO:dedupe.canopy_index:Removing stop word de
INFO:dedupe.canopy_index:Removing stop word ei
INFO:dedupe.canopy_index:Removing stop word er
INFO:dedupe.canopy_index:Removing stop word hn
INFO:dedupe.canopy_index:Removing stop word le
INFO:dedupe.canopy_index:Removing stop word me
INFO:dedupe.canopy_index:Removing stop word or
INFO:dedupe.canopy_index:Removing stop word rs
INFO:dedupe.canopy_index:Removing stop word st
INFO:dedupe.canopy_index:Removing stop word  B
INFO:dedupe.canopy_index:Removing stop word St
INFO:dedupe.canopy_index:Removing stop word at
INFO:dedupe.canopy_index:Removing stop word dt
INFO:dedupe.canopy_index:Removing stop word he
INFO:dedupe.canopy_index:Removing stop word ie
INFO:dedupe.canopy_index:Removing stop word is
INFO:dedupe.canopy_index:Removing stop word ra
INFO:dedupe.c

In [38]:
len(clustered_dupes)

13977

In [39]:
classes = list()
for i in range(len(ks)):
    classes.append(-1)

In [40]:
clustered_dupes[0]

((3, 495), (0.9824463, 0.9824463))

In [41]:
for r in clustered_dupes:
    i = clustered_dupes.index(r)
    #print(i, ' : ')
    for e in r[0]:
        #print(e)
        classes[e] = i

In [42]:
dfk = df_kun.copy(deep=True)
#df_kun = dfk


In [43]:
d_kun = pd.merge(df_kun, pd.Series(classes, name='Dublettengruppe'), how='left', left_index=True, right_index=True)

In [44]:
d_kun.to_excel('data/marked_doublette_groups.xlsx')